In [3]:
import pandas as pd
import geopandas as gpd
import requests

# Data Preprocessing

### Read data

In [2]:
token = "vAtl2Z7zE587vSL1K0jPGUqgw"

Download zipcode data

In [3]:
df = gpd.read_file("nyc_zipcodes.shp")
raw_zipcode = pd.DataFrame(df)
raw_zipcode.head()

DriverError: nyc_zipcodes.shp: No such file or directory

Download zillow data

In [ ]:
raw_zillow = pd.read_csv("zillow_rent_data.csv")
raw_zillow.head()

Download 311 data with API

In [4]:
# API endpoint URL
url_311 = 'https://data.cityofnewyork.us/resource/erm2-nwe9.json'

# Set up the headers with the API token
headers = {'X-App-Token': token}

# Make the API request
response = requests.get(url_311, headers=headers)
data = response.json()

# Print the data
raw_311= pd.DataFrame(data)
raw_311.head()

,unique_key,created_date,agency,agency_name,complaint_type,descriptor,location_type,intersection_street_1,intersection_street_2,address_type,...,:@computed_region_7mpf_4k6g,landmark,vehicle_type,taxi_pick_up_location,bridge_highway_name,road_ramp,bridge_highway_segment,bridge_highway_direction,facility_type,taxi_company_borough
0,59589245,2023-11-30T12:00:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,CLYMER STREET,CLYMER STREET,INTERSECTION,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,59589235,2023-11-30T12:00:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,NaN,NaN,ADDRESS,...,67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,59587554,2023-11-30T01:21:01.000,NYPD,New York City Police Department,Illegal Parking,Double Parked Blocking Vehicle,Street/Sidewalk,GROVE STREET,CYPRESS AVENUE,ADDRESS,...,62,GROVE STREET,SUV,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,59593739,2023-11-30T01:20:51.000,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,19 STREET,20 STREET,ADDRESS,...,45,6 AVENUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,59593633,2023-11-30T01:20:19.000,NYPD,New York City Police Department,Blocked Driveway,Partial Access,Street/Sidewalk,107 AVENUE,SHORE AVENUE,ADDRESS,...,61,WALTHAM STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Download tree data with API

In [5]:
# API endpoint URL
url_tree = 'https://data.cityofnewyork.us/resource/5rq2-4hqu.json?$limit=100000000'

# Set up the headers with the API token
headers = {'X-App-Token': token}

# Make the API request
response = requests.get(url_tree, headers=headers)
data = response.json()

# Print the data
raw_tree = pd.DataFrame(data)
raw_tree.head()

,created_at,tree_id,block_id,the_geom,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,...,st_assem,st_senate,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp
0,08/27/2015,180683,348711,"{'type': 'Point', 'coordinates': [-73.84421521...",3,0,OnCurb,Alive,Fair,Acer rubrum,...,28,16,QN17,Forest Hills,4073900,New York,40.72309177,-73.84421522,1027431.14821,202756.768749
1,09/03/2015,200540,315986,"{'type': 'Point', 'coordinates': [-73.81867945...",21,0,OnCurb,Alive,Fair,Quercus palustris,...,27,11,QN49,Whitestone,4097300,New York,40.79411067,-73.81867946,1034455.70109,228644.837379
2,09/05/2015,204026,218365,"{'type': 'Point', 'coordinates': [-73.93660770...",3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,50,18,BK90,East Williamsburg,3044900,New York,40.71758074,-73.9366077,1001822.83131,200716.891267
3,09/05/2015,204337,217969,"{'type': 'Point', 'coordinates': [-73.93445615...",10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,53,18,BK90,East Williamsburg,3044900,New York,40.71353749,-73.93445616,1002420.35833,199244.253136
4,08/30/2015,189565,223043,"{'type': 'Point', 'coordinates': [-73.97597938...",21,0,OnCurb,Alive,Good,Tilia americana,...,44,21,BK37,Park Slope-Gowanus,3016500,New York,40.66677776,-73.97597938,990913.775046,182202.425999


### Removing unnecessary columns

In [ ]:
df_z = raw_zillow.copy()
df_zillow_1 = pd.concat([df_z.iloc[:, [2, 6]], df_z.iloc[:, 9:]], axis=1)
df_zillow_1.head()

In [ ]:
df_3 = raw_311.copy()
df_311_1 = df_3.loc[:, ["created_date", "complaint_type", "incident_zip", "latitude", "longitude"]]
df_311_1.head()

In [ ]:
df_t = raw_tree.copy()
df_tree_1 = df_t.loc[:, ["zipcode", "latitude", "longitude", "x_sp", "y_sp"]]
df_tree_1.head()

### Remove invalid data points 

Drop the rows in df_zillow in which city is not New York

In [ ]:
index = df_zillow_1[df_zillow_1['City'] != "New York"].index
df_zillow_2 = df_zillow_1.drop(index).reset_index(drop=True)
df_zillow_2 = df_zillow_2.drop(["City"], axis=1)
df_zillow_2.head()

### Normalize column names & column types where needed

Normalize the name of zipcode colunms

### Normalize the Spatial Reference Identifiers (SRID) of any geometry.

# Storing Data

In [1]:
!createdb 4501_Final_Project

createdb: error: database creation failed: ERROR:  database "4501_Final_Project" already exists


In [2]:
!psql --dbname 4501_Final_Project -c 'CREATE EXTENSION postgis;'

ERROR:  extension "postgis" already exists


# Understanding Data

# Visualizing Data